In [ ]:
import pystan
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.spatial import distance
import numpy as np
import arviz as az
from classes.mapping import neighborhood_mapping
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import split
from pointpats import PointPattern
from pointpats import PoissonPointProcess as csr
from pointpats import distance_statistics as ds

In [ ]:
# Loading the data

# Shapefiles
shapefile_path = 'data/neighborhoods'
sf_neighborhoods = gpd.read_file(shapefile_path)

# SF case and population data
sf_cases = pd.read_csv('data/master_merged.csv', index_col=False)
sf_cases.rename(columns={'Neighborhood': 'nhood'}, inplace=True)

In [ ]:
sf_neighborhoods.loc[4]['geometry']

In [ ]:
sf_neighborhoods.to_csv('data/sf_neighborhood_geometry.csv')

In [ ]:
specific_neighborhood = sf_neighborhoods[sf_neighborhoods['nhood'] == 'South of Market']

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
specific_neighborhood.plot(ax=ax, legend=True, cmap='cividis')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

In [ ]:
specific_neighborhood.geometry

In [ ]:
cases = pd.read_csv('data/encampment_cases_filtered.csv')
cases.dropna(subset=['Neighborhood'], inplace=True)
cases.drop(columns='Unnamed: 0', inplace=True)
cases['Mapped Neighborhood'] = cases['Neighborhood'].map(neighborhood_mapping)

In [ ]:
som_cases = cases[cases['Mapped Neighborhood'] == 'South of Market']
som_cases

In [ ]:
som_cases['geometry'] = som_cases['Point'].apply(lambda x: Point(float(x.split(',')[1][1:-1]), float(x.split(',')[0][1:])))

gdf_points = gpd.GeoDataFrame(som_cases, geometry='geometry')

ax = specific_neighborhood.plot(cmap='cividis', edgecolor='black')

gdf_points.plot(ax=ax, color='orange', markersize=5)

plt.show()

In [ ]:
coordinates = [
    (-122.398352, 37.779579), (-122.399629, 37.780601), 
    (-122.401595, 37.779054), (-122.403866, 37.776978), 
    (-122.404450, 37.776602), (-122.406102, 37.775476), 
    (-122.406639, 37.774996), (-122.408340, 37.773780), 
    (-122.407023, 37.772736), (-122.406463, 37.773097), 
    (-122.405979, 37.773475), (-122.404982, 37.774278), 
    (-122.403281, 37.775634), (-122.400326, 37.778103)
]

polygon = Polygon(coordinates)

polygon_gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs=gdf_points.crs)


In [ ]:
neighborhood_polygon = specific_neighborhood.unary_union

points_within_neighborhood = gdf_points.within(neighborhood_polygon)

gdf_points_within = gdf_points[points_within_neighborhood]

ax = specific_neighborhood.plot(cmap='cividis', edgecolor='black')
gdf_points_within.plot(ax=ax, color='orange', markersize=5)
polygon_gdf.boundary.plot(ax=ax, color='red', linewidth=2) 

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
gdf_points_within.to_csv('data/som_cases.csv')

In [ ]:
line_coords = [
    (-122.398732, 37.787138),
    (-122.404450, 37.776602),
    (-122.406102, 37.775476),
    (-122.406639, 37.774996),
    (-122.408340, 37.773780),
    (-122.418126, 37.771588),
]

line = LineString(line_coords)

polygon = specific_neighborhood.geometry.unary_union

split_polygons = split(polygon, line)

upper_polygon = split_polygons.geoms[0]
upper_gdf = gpd.GeoDataFrame({'geometry': [upper_polygon]}, crs=specific_neighborhood.crs)

In [ ]:
fig, ax = plt.subplots()
specific_neighborhood.plot(ax=ax, color='lightgrey', edgecolor='black') 
upper_gdf.plot(ax=ax, color='blue', edgecolor='black')  
plt.show()

In [ ]:
upper_gdf_polygon = upper_gdf.unary_union

points_within_upper_polygon = gdf_points.within(upper_gdf_polygon)

gdf_points_within2 = gdf_points[points_within_upper_polygon]

ax = upper_gdf.plot(cmap='cividis', edgecolor='black')
gdf_points_within2.plot(ax=ax, color='orange', markersize=5)

# Hospitality House / city team / self-help center
specific_point = gpd.GeoDataFrame({'geometry': [Point(-122.407483, 37.780046)]}, crs=gdf_points.crs)
specific_point.plot(ax=ax, color='red', markersize=50) 

# A Woman's Place / veteran care/ rise
specific_point = gpd.GeoDataFrame({'geometry': [Point(-122.408502, 37.778684)]}, crs=gdf_points.crs)
specific_point.plot(ax=ax, color='red', markersize=50) 

# The Sanctuary
specific_point = gpd.GeoDataFrame({'geometry': [Point(-122.411548, 37.776242)]}, crs=gdf_points.crs)
specific_point.plot(ax=ax, color='red', markersize=50) 

# Salvation army / Shelter Client Advocates
specific_point = gpd.GeoDataFrame({'geometry': [Point(-122.409141, 37.781345)]}, crs=gdf_points.crs)
specific_point.plot(ax=ax, color='red', markersize=50) 

In [ ]:
shelter_coords = [
    (-122.407483, 37.780046),
    (-122.408502, 37.778684),
    (-122.411548, 37.776242),
    (-122.409141, 37.781345)
]

shelters = gpd.GeoDataFrame({
    'shelter_id': range(len(shelter_coords)), 
    'geometry': [Point(lon, lat) for lon, lat in shelter_coords]
}, crs=gdf_points_within2.crs)

cases_with_nearest_shelter = gpd.sjoin_nearest(gdf_points_within2, shelters, distance_col="distance")

cases_with_nearest_shelter['distance_meters'] = cases_with_nearest_shelter['distance'] * 1000

In [ ]:
cases_with_nearest_shelter['distance_meters'].hist(bins=30)
plt.title('Histogram of Distances to Nearest Shelter')
plt.xlabel('Distance (meters)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
upper_gdf.to_file("data/study_region.geojson", driver="GeoJSON")

In [ ]:
cases_with_nearest_shelter.to_file("data/cases_with_distances.geojson", driver="GeoJSON")

In [ ]:
shelters.to_file("data/shelters.geojson", driver="GeoJSON")